# 작성자 추가

- 다대일 관계
    - 여러 개의 모델이 하나의 모델에 연결되는 관계
        - 블로그의 사용자는 각각 여러 개의 포스트를 작성할 수 있음
        - 각 포스트는 작성자 정보를 담을 수 있는 필드에 한 명의 사용자 정보만 담을 수 있음
            - 즉, 포스트와 작성자의 관계는 다대일 관계
        - 카테고리 정보를 추가하는 경우도 마찬가지
        - 하나의 카테고리에는 여러 개의 포스트가 포함될 수 있음
        - 반면 하나의 포스트는 하나의 카테고리만 지정 가능
            - 이런 경우 다대일 관계가 성립
            
- author 필드에 담을 내용
    - 사용자명을 문자열로 저장
    - 사용자가 사용자명을 바꾼다면 이전에 작성한 글의 작성자명도 함께 바뀌어야함
    - 탈퇴하거나 글을 삭제하면 작성자명을 "unknown"으로 표시하는 기능 필요

## ForeignKey로 author 필드 구현

- User 모델 사용
    - User는 장고에서 기본적으로 제공하는 모델
    - on_delete-models.CASCADE : 이 포스트의 작성자가 데이터베이스에서 삭제되었을 때 이 포스트도 함께 삭제한다

### 다른 사용자가 게시글을 작성하는 경우 테스트

- 관리자페이지에서 새로운 사용자를 만들기
    - Users 옆의 add버튼 클릭

### 작성자 정보가 삭제될 때 포스트 정보가 삭제되는지 확인

- models.py의 author 필드에서 on_delete=models.CASCADE로 지정했기 때문에 trump 사용자가 삭제될 때 trump가 작성한 포스트까지 다 삭제되어야 함

### 연결된 사용자가 삭제되면 빈 칸으로 두기

- 사용자가 삭제되어도 그 사용자가 작성한 글은 남겨두고 author 필드 값만 null로 바뀌도록 설정
    - on_delete=models.SET_NULL : 이 포스트의 작성자가 데이터베이스에서 삭제되었을 때 작성자 명을 빈 칸으로 둔다

### 포스트 작성자가 삭제되었을 때 테스트

- 새로운 사용자를 생성한 후 해당 사용자의 이름으로 새로운 포스트를 추가하고 삭제

## 포스트 목록 페이지와 포스트 상세 페이지에 author 반영하기

- 작성자 정보가 포스트 목록과 상세 페이지에 나타나도록 변경

### 테스트 코드 작성

- blog/tests.py에 from django.contrib.auth.models improt User 로 User 모델을 임포트
- setUp() 함수에 User 데이터를 2개 생성

- test_post_list() 함수에서 Post 모델의 post_001과 post_002데이터를 생성할 때 각각 author=self.user_trump와 author=self.user_biden을 인자로 추가

- 테스트 실패 이유는 TRUMP 라는 문구가 메인 영역에서 보이지 않았기 때문
    - post_list.html 파일을 수정해 작성자명을 입력

### 포스트 상세 페이지에 작성자 추가

- test_post_detail() 함수도 test_post_list() 함수와 마찬가지로 post_001을 생성할 때 작성자명을 포함하도록 수정
- 포스트의 작성자가 화면에 출력되는지 확인하는 코드 추가

# 카테고리 기능 구현

- 블로그 포스트 내용에 따라 카테고리를 구분해서 방문자가 원하는 분야의 글만 읽을 수 있도록 카테고리 기능 추가
- 현재의 템플릿에 카테고리 기능의 모양은 갖춰져 있음
    - 실제로 개발중인 사이트에 맞게 수정하면 됨
    
- 카테고리 페이지 구성
    - 페이지 오른쪽에 있는 Categories 카드에 블로그의 ㄹ카테고리가 모두 표시
    - 포스트 목록 페이지와 포스트 상세 페이지에 모두 나타나야함
    - 이 중 하나를 클릭하면 해당 카테고리 페이지로 이동
    - 카테고리 페이지에는 해당 카테고리에 들어 있는 포스트가 모두 나열
    - 각 포스트는 하나의 카테고리만 지정 가능
        - 카테고리는 여러 개의 포스트가 포함될 수 있으므로 포슽와 카테고리는 포슽와 작성자 처럼 다대일 관계
        - ForeignKey로 구현 가능
    - Category 모델은 직접 개발

## Category 모델 개발

- blog/models.py 에 Category 모델 추가

- name
    - 각 카테고리의 이름을 담는 필드 
    - unique=True : 동일한 name을 갖는 카테고리를 또 만들 수 없음
        - Post 모델의 title에서는 unique=True 를 설정하지 않았기 때문에 같은 제목을 갖는 블로그 포스트를 여러 개 만들 수 있음
- slug
    - SlugField : 사람이 읽을 수 있는 텍스트로 고유 URL을 만들고 싶을 때 사용
    - Post모델처럼 PK를 활용해 URL을 만들 수도 있지만, 카테고리는 포스트만큼 개수가 많지 않을것이기 때문에 사람이 읽고 그 뜻을 알 수 있게 고유 URL을 사용
    - 길이는 최대 200으로 제한하고, unique=True를 사용해 서로 다른 카테고리가 동일한 slug를 가질 수 없도록 설정
    - 기본적으로 SLugField는 한글을 지원하지 않지만 allow_unicode=True 로 설정해 한글로도 만들 수 있게 함

### post 모델에 category 필드 추가

- 카테고리가 미분류인 포스트도 있을 수 있으므로 null=True
- ForeignKey로 연결되어 있던 카테고리가 삭제된 경우 연결된 포스트까지 삭제되지 않고 해당 포스트의 category 필드만 null이 되도록 on_delete=models.SET_NULL 로 설정

### admin.py에 Category 모델 등록

- blog/admin.py에서 Category 모델을 임포트
- CategoryAdmin 클래스 추가 후 name 필드에 값이 입력됐을 때 자동으로 slug가 만들어지도록 설정

### Meta로 모델의 복수형 설정 

- 장고 Category의 복수형을 Categorys라고 썼음
- models.py에서 Category 모델의 메타 설정에서 verbose_name_plural을 추가해 복수형을 직접 설정

## 카테고리를 빈 칸으로 남겨두기

- Categoies 데이터를 추가
- Name에 이름을 입력하면 자동으로 특수기호나 빈 칸 등 URL로 사용하기 적합하지 않은 문자를 적절하게 변환한 다음 slug에 채워줌
    - CategoryAdmin 클래스의 기능